In [1]:
import pandas as pd
from dataprep.clean import *
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from optuna import create_study, visualization
from optuna.pruners import SuccessiveHalvingPruner
from optuna.samplers import RandomSampler
import shap
import matplotlib.pyplot as plt

import machine_learning_pipeline as mlp
import exploratory_data_analysis as eda
%load_ext autoreload
%autoreload 2

ModuleNotFoundError: No module named 'dataprep'

### Data Wrangling

#### Loading train and test dfs

In [ ]:
train_df = pd.read_csv('train_ab.csv')
test_df = pd.read_csv('test_ab.csv')

# cleaning the headers
train_df = clean_headers(train_df)
# confirming the id is unique for each row and setting it 
# as the index
assert train_df.id.nunique() == len(train_df)
train_df = train_df.set_index('id')

test_df = clean_headers(test_df)
assert test_df.id.nunique() == len(test_df)
test_df = test_df.set_index('id')

train_df

#### Handling the Target/Dependent Variable

In [ ]:
# assigning a column of the dataframe as the tarfet
target = 'rings'
# defining the type of our target: 'continuous', 'binary' or 'multiclass'
target_type = 'continuous'

# checking that the target_type is a valid one
assert target_type in ['continuous', 'binary', 'multiclass']

# encoding the target if binary or multi-class type is chosen
if target_type != 'continuous':
    le = LabelEncoder()
    train_df[target] = le.fit_transform(train_df[target].tolist())
    print('Target Encoding:')
    for i, clss in enumerate(list(le.classes_)): print(target + ' ' + clss + ' -> ' + target + ' ' + str(i)) 


#### Splitting features into numerical and categorical types

In [ ]:
# creating a dataframe that lets us know the data types of each one of our 
# features
data_types = pd.DataFrame(train_df.dtypes, columns=['feature_type'])
data_types['unique_values'] = train_df.nunique()
data_types

Although all the columns are numeric, the small number of diffetent values for some columns suggests that the they could be interpreted as categorical

In [ ]:
# setting a threshold to determine categorical columns versus numerical
cat_threshold = 10
numerical_features, categorical_features = eda.split_features(df=train_df, target_col=target, categorical_threshold=cat_threshold)

### EDA

#### Pairplot
These graphs help us visualize the relationships between our numerical features and the target. For numerical targets the last row of the pairplot corresponds to the target; for categorical targets, the target can be seen as the marker color in each graph.

In [ ]:
eda.pairplot(df=train_df,
             numerical_features=numerical_features,
             target_type=target_type,
             target_col=target,
             sample=0.5)

#### Numerical Features Distribution Comparison
The graphs below are useful to identify any major differences between our train and test sets that can impact our models. It also shows the general distribution of each of the numerical features

In [ ]:
eda.train_test_distribution_plots(train_df,
                                  test_df,
                                  numerical_features,
                                  sample=0.5)

#### Categorical Features Distribution Comparison
The graphs below are useful to identify any major differences between our train and test sets that can impact our models. It also shows the general distribution of each of the cateorical features

In [ ]:
eda.train_test_categorical_piecharts(train_df,
                                     test_df,
                                     categorical_features)

#### Correlation Matrix

In [ ]:
eda.correlation_plot(train_df[numerical_features + [target]])

### Model Training with Optuna

#### Declaring variables needed for optimization step

In [ ]:
# splitting the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(train_df.drop(target, axis=1), 
                                                    train_df[target], 
                                                    test_size=0.2, 
                                                    random_state=42)

In [ ]:
# During the model fitting step, the optuna optimizer will include the algorithm 
# as one of the  hyperparameters of the model. It will try to find the one that 
# performs the best for our problem.

if target_type == 'continuous':    
    optimization_objective = 'regression'
    # available algorithms for regression tasks. 
    algorithms = [
                    # 'linear', 
                    # 'ridge',
                    # 'histgb',                     
                    # 'extratrees', 
                    'lgb',
                    # 'xgb', 
                    # 'catboost'
                ] 
    
    # scoring for cross-validation
    optimization_scoring = 'r2'

elif target_type == 'binary':
    optimization_objective = 'classification'
    # available algorithms for binary classification tasks
    algorithms = [
                    # 'histgb',                     
                    # 'extratrees', 
                    'lgb',
                    # 'xgb', 
                    # 'catboost'
                ]
    # scoring for cross-validation
    optimization_scoring = 'roc_auc'

elif target_type == 'multiclass':
    optimization_objective = 'multiclass'
    # available algorithms for multiclass classification tasks
    algorithms = [
                    # 'histgb',                     
                    'lgb',
                ]
    # scoring for cross-validation
    optimization_scoring = 'neg_log_loss'

#### Model Optimization

In [ ]:
%%time
# declaring our optuna study
factor = 2
# the pruner parameter helps make the optimization much faster as it will prune
# any iterations that don't look promising right from the start
study = create_study(study_name='optimization', 
                     direction='maximize',
                     pruner=SuccessiveHalvingPruner(reduction_factor=factor),
                     sampler=RandomSampler(seed=0))


# custom function to fit the model using smart hyperparameter search with optuna. This 
# parameter search is not restricted to the algorithm's hyperparameters. The algorithm
# itself is part of the parameters to be optimized. Data preprocessing decisions are 
# also included here, such as the scaling approach for numerical features, the encoding 
# technique for categorical variables, as well as feature selection.
study.optimize(lambda trial: mlp.objective(trial, 
                                           X_train, 
                                           y_train, 
                                           objective=optimization_objective,
                                           algorithms=algorithms,
                                           cv_scoring=optimization_scoring,
                                           numerical_columns=numerical_features, 
                                           categorical_columns=categorical_features,
                                           base=factor, 
                                           n_rungs=4), 
               n_trials=10)

In [ ]:
# saving the models best parameters and trial and printing its best score
best_model_params = study.best_params
best_trial = study.best_trial
# printing the score of the best model
print(study.best_value)

#### Optuna trials visualizations

In [ ]:
visualization.plot_optimization_history(study)

In [ ]:
visualization.plot_param_importances(study, target_name="ms")

#### Instantiating the model

In [ ]:
# calling the best trial and instantiating the model
model = mlp.instantiate_model(best_trial, 
                              numerical_features, 
                              categorical_features, 
                              optimization_objective, 
                              algorithms)
model.fit(X_train, y_train)

In [ ]:
# printing score on the test set
print(model.score(X_test,y_test))

### Model Explainability with SHAP

#### Fitting the model on preprocessed data

In [ ]:
best_model_params

#### Applying preprocessing steps to our dataset
To be able to compute the shap values we need the preprocessed dataset and the chosen model. We'll use the best_model_params dictionary to retrieve them

In [ ]:
from lightgbm import LGBMClassifier, LGBMRegressor
from sklearn.ensemble import RandomForestClassifier, ExtraTreesRegressor, HistGradientBoostingRegressor, AdaBoostRegressor
from sklearn.linear_model import LogisticRegression, LinearRegression
from category_encoders import OrdinalEncoder, OneHotEncoder, WOEEncoder, TargetEncoder, CatBoostEncoder, SumEncoder, BinaryEncoder, HelmertEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import RobustScaler
from sklearn.compose import ColumnTransformer


# numerical_features_used = [col for col in numerical_features if best_model_params[col]]
numerical_features_used = numerical_features
# categorical_features_used = [col for col in categorical_features if best_model_params[col]]
categorical_features_used = categorical_features

X_train_preprocessed = X_train[numerical_features_used + categorical_features_used]
X_test_preprocessed = X_test[numerical_features_used + categorical_features_used]

valid_scaler_params = ['with_centering', 'with_scaling']
# Filter out non-preprocessing hyperparameters
preprocessing_params = {k: v for k, v in best_model_params.items() if k in valid_scaler_params}
final_scaler = RobustScaler(**preprocessing_params)

encoder_strategy = best_model_params['categorical_encoder']
if encoder_strategy == 'ordinal':
    encoder = OrdinalEncoder()
elif encoder_strategy == 'onehot':
    encoder = OneHotEncoder()
elif encoder_strategy == 'binary':
    encoder = BinaryEncoder()
elif encoder_strategy == 'helmert':
    encoder = HelmertEncoder()
elif encoder_strategy == 'sum':
    encoder = SumEncoder()
elif encoder_strategy == 'target':
    encoder = TargetEncoder()
elif encoder_strategy == 'woe':
    encoder = WOEEncoder()
elif encoder_strategy == 'catboost':
    encoder = CatBoostEncoder()
else:
    encoder = ''

final_processor = ColumnTransformer([
    ('scaler', final_scaler, numerical_features_used),
    ('encoder', encoder, categorical_features_used)
  ])

final_model_params = {k: v for k, v in best_model_params.items() if k not in (valid_scaler_params+['categorical_encoder'])}
del final_model_params['algorithm']
# for col in X_train.columns.to_list():
#     del final_model_params[col]
# final_model_params['objective'] = 'multiclass'
# final_model = HistGradientBoostingRegressor(**final_model_params)
final_model = LGBMRegressor(**final_model_params)
# final_model = LinearRegression(**final_model_params)

X_train_preprocessed = final_processor.fit_transform(X_train_preprocessed, y_train)
X_train_preprocessed = pd.DataFrame(X_train_preprocessed, columns=numerical_features + categorical_features, index=X_train.index)

X_test_preprocessed = final_processor.fit_transform(X_test_preprocessed, y_test)
X_test_preprocessed = pd.DataFrame(X_test_preprocessed, columns=numerical_features + categorical_features, index=X_test.index)


final_model.fit(X_train_preprocessed, y_train)

#### Calculating Shap Values

In [ ]:
explainer = shap.TreeExplainer(final_model)
# explainer = shap.Explainer(final_model, X_test_preprocessed)
shap_values = explainer.shap_values(X_train_preprocessed, y_train)

In [ ]:
# making sure the labels are correct
data_types = pd.DataFrame(X_train_preprocessed.dtypes, columns=['feature_type'])
data_types['unique_values'] = X_train_preprocessed.nunique()
data_types

#### Shap summary plot

In [ ]:
if target_type != 'multiclass':     # graph not supported for multiclass
    shap.summary_plot(shap_values, 
                      X_train_preprocessed, 
                      feature_names=X_train_preprocessed.columns, 
                      plot_type='dot')
else:
    shap.summary_plot(shap_values, 
                      X_train_preprocessed, 
                      plot_type="bar", 
                      feature_names = X_train_preprocessed.columns)

    for i, tclass in enumerate(list(le.classes_)):
        shap.summary_plot(shap_values[i], 
                  X_train_preprocessed, 
                  plot_type="dot", 
                  feature_names = X_train_preprocessed.columns,
                  show=False) 
        plt.title(f'Shap values for {target} {tclass}')
        plt.show() 


#### Partial Dependence Plots

In [ ]:
if target_type == 'multiclass':
    # producing one graph for each class
    for i, tclass in enumerate(le.classes_):
        mlp.shap_partial_dependence_plots(X_train_preprocessed, shap_values[i]) 
else:
    mlp.shap_partial_dependence_plots(X_train_preprocessed, shap_values)

Takeaways from the previous graphs: 

### Model Predictions Analysis

#### Train Set

In [ ]:
if target_type == 'continuous':
    train_results = mlp.append_predictions(model=model, 
                                           df=X_train, 
                                           target_values=y_train,
                                           target_name=target,
                                           target_type=target_type, 
                                           df_preprocessed=X_train_preprocessed)
else:
    train_results = mlp.append_predictions(model=model, 
                                           df=X_train, 
                                           target_values=y_train,
                                           target_name=target,
                                           target_type=target_type, 
                                           df_preprocessed=X_train_preprocessed, 
                                           label_encoder=le)

train_results

##### Overall Observations vs. Predictions plot

In [ ]:
if target_type != 'continuous':
    target_classes = list(le.classes_)
    mlp.confusion_matrix_plot(y_true=train_results[target],
                              y_pred=train_results[target + '_prediction'],
                              labels=target_classes)

In [ ]:
import plotly.express as px

if target_type == 'continuous':
    hover_col = 'id'
    fig = px.scatter(train_results, 
                     x=target, 
                     y=f"{target}_prediction", 
                     hover_data=[hover_col],
                     width=600,
                     height=600)
    fig.show()
else:
    mlp.prediction_probability_distribution_plot(preds_df=train_results, 
                                                 target_classes=target_classes, 
                                                 target_colname=target)


##### Single prediction waterfall plot

In [ ]:
shap_explainer_values_train = explainer(X_train_preprocessed, y_train)

In [ ]:
datapoint_id = 26481
idx = train_results[train_results.id == datapoint_id].index[0]
if target_type != 'multiclass':
    shap.waterfall_plot(shap_explainer_values_train[idx])
else:
    for i, tclass in enumerate(target_classes):
        shap.waterfall_plot(shap_explainer_values_train[idx][:,i], show=False)
        plt.title(f'Shap values for {target} {tclass}')
        plt.show()

In [ ]:
fig = px.box(train_results,
             x=target, 
             y='pred_prob_CL',
             hover_data=["id"],
             width=600,
             height=600)
fig.show()

#### Test Set

In [ ]:
if target_type == 'continuous':
    test_results = mlp.append_predictions(model=model, 
                                          df=X_test, 
                                          target_values=y_test,
                                          target_name=target,
                                          target_type=target_type, 
                                          df_preprocessed=X_test_preprocessed)
else:
    test_results = mlp.append_predictions(model=model, 
                                          df=X_test, 
                                          target_values=y_test,
                                          target_name=target,
                                          target_type=target_type, 
                                          df_preprocessed=X_test_preprocessed, 
                                          label_encoder=le)

test_results

##### Overall Observations vs. Predictions plot

In [ ]:
if target_type != 'continuous':
    target_classes = list(le.classes_)
    mlp.confusion_matrix_plot(y_true=test_results[target],
                              y_pred=test_results[target + '_prediction'],
                              labels=target_classes)

In [ ]:
if target_type == 'continuous':
    hover_col = 'id'
    fig = px.scatter(test_results, 
                     x=target, 
                     y=f"{target}_prediction", 
                     hover_data=[hover_col],
                     width=600,
                     height=600)
    fig.show()
else:
    mlp.prediction_probability_distribution_plot(preds_df=test_results, 
                                                 target_classes=target_classes, 
                                                 target_colname=target)


In [ ]:
fig = px.box(test_results,
             x=target, 
             y='pred_prob_C',
             hover_data=["id"],
             width=600,
             height=600)
fig.show()

##### Single prediction waterfall plot

In [ ]:
shap_explainer_values_test = explainer(X_test_preprocessed, y_test)

In [ ]:
datapoint_id = 42385
idx = test_results[test_results.id == datapoint_id].index[0]
if target_type != 'multiclass':
    shap.waterfall_plot(shap_explainer_values_test[idx])
else:
    for i, tclass in enumerate(target_classes):
        shap.waterfall_plot(shap_explainer_values_test[idx][:,i], show=False)
        plt.title(f'Shap values for {target} {tclass}')
        plt.show()

### Conclusion

In [ ]:
train_df_mod = train_df.copy()
train_df_mod['height_cm*hemoglobin'] = train_df_mod['height_cm']*train_df_mod['hemoglobin']
train_df_mod['height_cm*gtp'] = train_df_mod['height_cm']*train_df_mod['gtp']
train_df_mod['height_cm*triglyceride'] = train_df_mod['height_cm']*train_df_mod['triglyceride']
train_df_mod['age*hemoglobin'] = train_df_mod['age']*train_df_mod['hemoglobin']

X_train, X_test, y_train, y_test = train_test_split(train_df_mod.drop(target, axis=1), 
                                                    train_df_mod[target], 
                                                    test_size=0.2, 
                                                    random_state=42)

new_features = ['height_cm*hemoglobin', 'height_cm*gtp', 'height_cm*triglyceride', 'age*hemoglobin']

In [ ]:
# %%time
# from optuna.pruners import SuccessiveHalvingPruner
# from optuna.samplers import RandomSampler



# factor = 2
# study = create_study(study_name='optimization', 
#                      direction='maximize',
#                      pruner=SuccessiveHalvingPruner(reduction_factor=factor),
#                      sampler=RandomSampler(seed=0)
#                     )

# optimization_objective = 'classification'
# optimization_scoring = 'roc_auc'

# # algorithms = ['histgb', 'lgb', 'extratrees', 'xgb', 'catboost']
# algorithms = ['lgb']
# # algorithms = ['linear', 'ridge']

# study.optimize(lambda trial: mlp.objective(trial, 
#                                        X_train, 
#                                        y_train, 
#                                        objective=optimization_objective,
#                                        algorithms=algorithms,
#                                        cv_scoring=optimization_scoring,
#                                        numerical_columns=numerical_features + new_features, 
#                                        categorical_columns=categorical_features,
#                                        base=factor, 
#                                        n_rungs=4), 
#                n_trials=200)

In [ ]:
best_model_params = study.best_params
best_trial = study.best_trial

In [ ]:
print(study.best_value)

In [ ]:
from lightgbm import LGBMClassifier, LGBMRegressor
from sklearn.ensemble import RandomForestClassifier, ExtraTreesRegressor, HistGradientBoostingRegressor, AdaBoostRegressor
from sklearn.linear_model import LogisticRegression, LinearRegression
from category_encoders import OrdinalEncoder, OneHotEncoder, WOEEncoder, TargetEncoder, CatBoostEncoder, SumEncoder, BinaryEncoder, HelmertEncoder
from sklearn.preprocessing import LabelEncoder

# numerical_features_used = [col for col in numerical_features if best_model_params[col]]
numerical_features_used = numerical_features
# categorical_features_used = [col for col in categorical_features if best_model_params[col]]
categorical_features_used = categorical_features

X_train_preprocessed = X_train[numerical_features_used + categorical_features_used + new_features]
X_test_preprocessed = X_test[numerical_features_used + categorical_features_used + new_features]

valid_scaler_params = ['with_centering', 'with_scaling']
# Filter out non-preprocessing hyperparameters
preprocessing_params = {k: v for k, v in best_model_params.items() if k in valid_scaler_params}
final_scaler = RobustScaler(**preprocessing_params)

encoder_strategy = best_model_params['categorical_encoder']
if encoder_strategy == 'ordinal':
    encoder = OrdinalEncoder()
elif encoder_strategy == 'onehot':
    encoder = OneHotEncoder()
elif encoder_strategy == 'binary':
    encoder = BinaryEncoder()
elif encoder_strategy == 'helmert':
    encoder = HelmertEncoder()
elif encoder_strategy == 'sum':
    encoder = SumEncoder()
elif encoder_strategy == 'target':
    encoder = TargetEncoder()
elif encoder_strategy == 'woe':
    encoder = WOEEncoder()
elif encoder_strategy == 'catboost':
    encoder = CatBoostEncoder()
else:
    encoder = ''

final_processor = ColumnTransformer([
    ('scaler', final_scaler, numerical_features_used + new_features),
    ('encoder', encoder, categorical_features_used)
  ])

final_model_params = {k: v for k, v in best_model_params.items() if k not in (valid_scaler_params+['categorical_encoder'])}
del final_model_params['algorithm']
# for col in X_train.columns.to_list():
#     del final_model_params[col]

# final_model = HistGradientBoostingRegressor(**final_model_params)
final_model = LGBMRegressor(**final_model_params)
# final_model = LinearRegression(**final_model_params)

X_train_preprocessed = final_processor.fit_transform(X_train_preprocessed, y_train)
X_train_preprocessed = pd.DataFrame(X_train_preprocessed, columns=numerical_features_used + new_features + categorical_features_used, index=X_train.index)

X_test_preprocessed = final_processor.fit_transform(X_test_preprocessed, y_test)
X_test_preprocessed = pd.DataFrame(X_test_preprocessed, columns=numerical_features_used + new_features + categorical_features_used, index=X_test.index)


final_model.fit(X_train_preprocessed, y_train)

In [ ]:
data_types = pd.DataFrame(X_train_preprocessed.dtypes, columns=['feature_type'])
data_types['unique_values'] = X_train_preprocessed.nunique()
data_types

In [ ]:
explainer = shap.TreeExplainer(final_model)
# explainer = shap.Explainer(final_model, X_test_preprocessed)
shap_values = explainer.shap_values(X_train_preprocessed, y_train)

In [ ]:
shap.summary_plot(shap_values, 
                  X_train_preprocessed, 
                  feature_names=X_train_preprocessed.columns, 
                  plot_type='dot')